## Cargo la base de tiendas que tengo que analizar

In [45]:
import pandas as pd
from datetime import datetime

### Trabajo sobre el detalle de las ventas por Media Hora

In [46]:
# Cargo la Informacion de Ventas Historico en Formato Long
df = pd.read_csv('data/horario_extendido/venta por media hora.csv', encoding='utf-16', header=2)
# Genero Copia del DF
df = df.copy()
# Genero un Slicing
df = df[1:]
# Renombro algunas columnas para trabajar mas comodod
df = df.rename(columns={
    ' ':'año',
    ' .2':'fecha',
    ' .3':'hora'
})
# Elimino aquellas columnas que molestan
df = df.drop(columns=[' .1', 'Punto Operacional'])
# Derrito el df para trabajar en forto "Wide"
df = df.melt(id_vars=['año', 'fecha', 'hora'], var_name='tiendas', value_name='vct').dropna(subset=['vct'])
# Hago un reset Index y elimino el anterior que no estaba ordenado
df = df.reset_index().drop(columns=['index'])
# Transformo la columna de VCT a INT
df['vct'] = pd.to_numeric(df['vct'].str.replace('.', '').str.replace(',', '.'), errors='coerce')
# Genero una columna para trabajar con el mes
df['mes'] = df['fecha'].str.strip().str.split(' ').str[2].str.strip()
# Genero una columna para obtener el dia (Luego nos servira para filtrar unicamente los datos de los domingos)
df['dia'] = df['fecha'].str.strip().str.split(' ').str[0].astype(str)
# Genero una columna para obtener el NO (Numero Operacional) "ID tienda"
df['no'] = df['tiendas'].str.split(' ').str[0].astype(int)

# Genero listas auxiliares para confeccionar un diccionario de forma rapida y luego realizar un Mapeo
mes = 'Enero Febrero Marzo Abril Mayo Junio Julio Agosto Septiembre Octubre Noviembre Diciembre'.split(' ')
num_mes = '1 2 3 4 5 6 7 8 9 10 11 12'.split(' ')
mes_orden = dict(zip(mes, num_mes))

# Genero una columna auxiliar con el numero del mes para generar luego una fecha Parseada
df['mes_numerico'] = df['mes'].map(mes_orden)
df['fecha_parsed'] = df['mes_numerico'] + '/' + df['dia'] + '/' + df['año'].astype(str)

# Convierto la fecha parseada a DateTime y asi obtengo el detalle del dia de la semana para luego filtrar la informacion unicamente de los domigos
df['fecha_final'] = pd.to_datetime(df['fecha_parsed'], format='%m/%d/%Y')
# Obtengo el detalle del nombre del dia a partir de la columna generada anteriormente
df['nombre_dia'] = df['fecha_final'].dt.day_name()

# Genero un nuevo df UNICAMENTE con la informacion de los Domingos
df_domingos = df[df['nombre_dia'] == 'Sunday']
# Genero columna NO
df_domingos['no'] = df_domingos['tiendas'].str.split(' ').str[0].astype(str)

# Creamos columna mes-año
df_domingos['mes'] = df_domingos['fecha_final'].dt.month_name()
df_domingos['mes'] = df_domingos['mes'].astype(str).str.lower()

# Extraemos hora de inicio
df_domingos['hora_inicio'] = df_domingos['hora'].str.extract(r'Desde (\d{2}:\d{2})')[0]
# Convertimos a objeto time
df_domingos['hora_inicio'] = pd.to_datetime(df_domingos['hora_inicio'], format='%H:%M').dt.time

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_3404\3255924576.py:2: DtypeWarning: Columns (0,4,13,106,458,459,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/horario_extendido/venta por media hora.csv', encoding='utf-16', header=2)
C:\Users\juan_mera\AppData\Local\Temp\ipykernel_3404\3255924576.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domingos['no'] = df_domingos['tiendas'].str.split(' ').str[0].astype(str)
C:\Users\juan_mera\AppData\Local\Temp\ipykernel_3404\3255924576.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

### Cargo el detalle de Costo Horas Hombre por Tienda

In [47]:
costo_horas_hombre = pd.read_excel('data/horario_extendido/padron_horario_extendido.xlsx', sheet_name='costo_ho')
# Elimino las columnas que estan en Nulo
costo_horas_hombre = costo_horas_hombre.dropna(axis=1)
# Genero un bucle para renombrar las columnas de forma correcta. En caso de que el nombre de la columna sea datetime, le coloca el nombre del mes en ingles en minuscula. Caso contrario, coloca el nombre en minuscula
for col in costo_horas_hombre.columns:
    if isinstance(col, datetime):
        costo_horas_hombre = costo_horas_hombre.rename(columns={
            col:pd.to_datetime(col).month_name().lower().strip()
        })
    else:
        costo_horas_hombre = costo_horas_hombre.rename(columns={
            col:col.lower().strip()
        })

# Derrito el DF para trabajar con el detalle de los meses como variable
costo_horas_hombre = costo_horas_hombre.melt(id_vars=['no', 'nombre', 'localidad', 'zona', 'gerente regional', 'domingos', 'horario_anterior'], value_name='costo_hora_hombre', var_name='mes')
# Multiplico los valores por -1 para convertir su valor a negativo
costo_horas_hombre['costo_hora_hombre'] = costo_horas_hombre['costo_hora_hombre'] * - 1

### Genero un DF Auziliar para contabilizar la cantidad de Domingos por Mes

In [48]:
# Contamos domingos por mes
domingos_por_mes = df_domingos.groupby('mes')['fecha_final'].nunique().reset_index()
domingos_por_mes.columns = ['mes', 'cantidad_domingos']

### Cargo el Margen

In [49]:
# Cargo el Margen. El formato de descarga de Tableau me tira el reporte con Separadores de ";". Verificar que esto en la computadora del trabajo sea igual
margen = pd.read_csv('data/horario_extendido/margen_comercial_acum.csv', sep=';')
# Pivoteo la informacion para colocar el "Rubro" es decir el Margen y la Venta sin Tasa, para asi calcular el Ratio Margen por tienda y finalmente llevarlo al DF principal
margen = margen.pivot_table(values='Importe Ars', columns='RUBRO_CONCAT (grupo) 1', index=['Periodo', 'Tienda'], aggfunc='sum').reset_index()
# Relleno valores nulos con ceros para no tener problemas a la hora de dividir valores
margen['VENTA SIN TASA'] = margen['VENTA SIN TASA'].fillna(0)
# Convierto los valores a Numericos antes de realizar su division
margen['MARGEN COMERCIAL'] = pd.to_numeric(margen['MARGEN COMERCIAL'].str.replace(',', '.')).round(2)
margen['VENTA SIN TASA'] = pd.to_numeric(margen['VENTA SIN TASA'].str.replace(',', '.')).round(2)
# Genero la columna de Ratio Margen
margen['ratio_margen'] = (margen['MARGEN COMERCIAL'] / margen['VENTA SIN TASA']).round(2)
# Relleno con ceros por las dudas aquellos valores nulos
margen = margen.fillna(0)
# genero una columna auxiliar para obtener el "numero del mes"
margen['mes_numerico'] = margen['Periodo'].astype(str).str.split('2025').str[1]
# Genero una columna auxiliar con el año
margen['año'] = '2025'
# Parseo todas estas columnas
margen['fecha_parsed'] = margen['mes_numerico'] + '/01/' + margen['año']
# Convierto la columna Parseada a Datetime
margen['fecha_final'] = pd.to_datetime(margen['fecha_parsed'], format='%m/%d/%Y')
# Obtengo el detalle del nombre del mes, en ingles. Este detalle, mas el NO, me servirá para realizar un Join entre este DF y el Principal
margen['mes'] = margen['fecha_final'].dt.month_name().str.lower()
# Convierto el NO en Numero
margen['Tienda'] = margen['Tienda'].astype(int)
# Renombro la columna para que tenga sentido
margen = margen.rename(columns={'Tienda':'no'})

### Cargo el detalle de Horas Autorizadas para las tiendas

In [ ]:
horas_autorizadas = pd.read_excel('data/horario_extendido/padron_horario_extendido.xlsx', sheet_name='horas_dom')
# Elimino las columnas que estan en Nulo
horas_autorizadas = horas_autorizadas.dropna(axis=1)
# Genero un bucle para renombrar las columnas de forma correcta. En caso de que el nombre de la columna sea datetime, le coloca el nombre del mes en ingles en minuscula. Caso contrario, coloca el nombre en minuscula
for col in horas_autorizadas.columns:
    if isinstance(col, datetime):
        horas_autorizadas = horas_autorizadas.rename(columns={
            col:pd.to_datetime(col).month_name().lower().strip()
        })
    else:
        horas_autorizadas = horas_autorizadas.rename(columns={
            col:col.lower().strip()
        })

# Derrito el df de LONG a Wide para colocar el detalle de los meses en las columnas y asi, comenzar a concatenar toda la informacion a este DF que sera el Final
horas_autorizadas = horas_autorizadas.melt(id_vars=['no', 'nombre', 'localidad', 'zona', 'gerente regional', 'domingos', 'horario_anterior'], value_name='horas', var_name='mes')
# Concateno el df Auxiliar con el detalle de la cantidad de domingos por mes al DF principal
horas_autorizadas = pd.merge(right=domingos_por_mes, left=horas_autorizadas, on='mes', how='left')
# Divido la cantidad de horas autorizadas del mes, por la cantidad de domingos para obtener la cantidad de Horas Autorizadas del Mes POR DOMINGO
horas_autorizadas['horas_por_domingo'] = round(horas_autorizadas['horas'] / horas_autorizadas['cantidad_domingos'],1)
# Genero un DF auziliar para calcular la Hora Promedio por Domingo del pormedio de Abril y Mayo (MESES ANTERIOR A REALIZAR LA EXTENSION HORARIA)
horas_prom_mayo_abril = horas_autorizadas[horas_autorizadas['mes'].isin(['april', 'may'])].groupby(['no'])['horas'].mean().reset_index().rename(columns={'horas':'horas_promedio_abril_y_mayo'})
horas_prom_mayo_abril['horas_promedio_abril_y_mayo'] = (horas_prom_mayo_abril['horas_promedio_abril_y_mayo'] / 4).round(2)
# Concateno el DF con las horas autorizadas para poder calcular luego la el crecimiento y diferencia de horario x Domingo por Tienda
horas_autorizadas = pd.merge(horas_autorizadas, horas_prom_mayo_abril, on='no', how='left')
horas_autorizadas['crecimiento_horas_domingo'] = (horas_autorizadas['horas_por_domingo'] / horas_autorizadas['horas_promedio_abril_y_mayo'] - 1).round(2)
horas_autorizadas['horas_adicionales_por_domingo'] = horas_autorizadas['horas_por_domingo'] - horas_autorizadas['horas_promedio_abril_y_mayo']

# Genero un DF Auxiliar para trabajar ahora sobre la hora de Apertura y Cierre que tenian las tiendas antes, para luego filtrar unicamente la venta fuera de esos horarios "Normales"
horarios_domingos = horas_autorizadas[['no', 'horario_anterior']].drop_duplicates()
# Separo el horario de cierre y apertura, luego lo convierto a TIME
horarios_domingos['horario_apertura'] = pd.to_datetime(horarios_domingos['horario_anterior'].str.split(' ').str[0], format='%H:%M', errors='coerce').dt.time
horarios_domingos['horario_cierre'] = pd.to_datetime(horarios_domingos['horario_anterior'].str.split(' ').str[2], format='%H:%M', errors='coerce').dt.time

# Me aseguro que su NO sea Int para realizar con exito un Merge entre el DF principal y asi colocarle a cada una de las tiendas, su horario de apertura y cierre convertido a DATETIME
horarios_domingos['no'] = horarios_domingos['no'].astype(int)
horas_autorizadas = pd.merge(horas_autorizadas, horarios_domingos[['no', 'horario_apertura', 'horario_cierre']], on='no', how='left')

# Me aseguro que el NO del DF donde coloque los horarios de las tiendas y el DF donde contiene la informacion de las ventas por media hora, Dia y Mes para realizar un MERGE. De esta forma Ahora no solamente voy a poder filtrar la informacion de los Domingos, sino tambien las ventas unicamente ocurridas entre el Horario Extendido
horarios_domingos['no'] = horarios_domingos['no'].astype(int)
df_domingos['no'] = df_domingos['no'].astype(int)
df_domingos = pd.merge(df_domingos, horarios_domingos[['no', 'horario_apertura', 'horario_cierre']], how='left', on='no').dropna(subset=['horario_apertura'])

# Genero el DF donde tengo solamente las ventas que necesito. Filtramos ventas fuera del horario normal (es decir, en horario extendido)
ventas_horario_extendido = df_domingos[(df_domingos['hora_inicio'] < df_domingos['horario_apertura']) | (df_domingos['hora_inicio'] >= df_domingos['horario_cierre'])]
# Ahora que tengo las ventas que quiero, unicamente me falta agruparlas para perder los detalles mas chicos. ORdeno estos valores tambien
ventas_horario_extendido = ventas_horario_extendido.groupby(['no', 'mes'])['vct'].sum().reset_index().sort_values(by='no')
# Concateno el DF que contiene la Venta Con tasa de los domingos, exclusivamente del horario extendido a las tiendas, teniendo en cuenta no solamente su NO sino el detalle del MES!
horas_autorizadas = pd.merge(horas_autorizadas, ventas_horario_extendido, on=['no', 'mes'], how='left')
# Relleno valores nulos con 0. Esto quiere decir que hay un conjunto de tiendas que NO tienen ventas en los horarios extendidos. Esto es precisamente porque hay tiendas que en los meses abril y mayo no tuvieron ventas
horas_autorizadas['vct'] = horas_autorizadas['vct'].fillna(0)
# Genero una nueva columna para calcular la VENTA SIN TASA, es decir, le quito los impuestos
horas_autorizadas['vst'] = horas_autorizadas['vct'] * (1 - 0.2650)

# Concateno el DF principal con el DF que contiene el detalle del ratio del margen por tienda y MES
horas_autorizadas = pd.merge(horas_autorizadas, margen[['no', 'mes', 'ratio_margen']], how='left', on=['no', 'mes'])
# Con la Ayuda del Ratio Margen y la Venta sin Tasa, calculo la Masa del Margen
horas_autorizadas['margen'] = (horas_autorizadas['vst'] * horas_autorizadas['ratio_margen']).round(2)
# calculo los gastos variables teniendo en cuenta la Venta sin Tasa
horas_autorizadas['gastos_variables'] = horas_autorizadas['vst'] * - 0.02
# Concateno el costo de horas hombre al DF principal
horas_autorizadas = pd.merge(horas_autorizadas, costo_horas_hombre[['no', 'mes', 'costo_hora_hombre']], on=['no', 'mes'], how='left')
# Calculo las horas adicionales totales por mes por tienda
horas_autorizadas['horas_adicionales_total_mes'] = horas_autorizadas['horas_adicionales_por_domingo'] * horas_autorizadas['cantidad_domingos']
# Multiplico la cantidad de horas adicionales por mes por el costo de horas hombre para obtener el costo total por mes por tienda
horas_autorizadas['gasto_de_personal'] = horas_autorizadas['horas_adicionales_total_mes'] * horas_autorizadas['costo_hora_hombre']
# Con el margen, los gastos variables y los gastos de personal, calculo el ROC en el horario extendido los dias Domingos
horas_autorizadas['roc'] = (horas_autorizadas['margen'] + horas_autorizadas['gastos_variables'] + horas_autorizadas['gasto_de_personal']).round(2)
# Renombro columnas para que tengan mas sentido
horas_autorizadas = horas_autorizadas.rename(columns={
    'mes':'mes_ing',
    'horas':'horas_autorizadas',
    'domingos':'horario_domingo',
})
# Genero un diccionario Auxiliar para traducir los meses de Ingles a Castellano
dic_meses = {
    'january': 'enero',
    'february': 'febrero',
    'march': 'marzo',
    'april': 'abril',
    'may': 'mayo',
    'june': 'junio',
    'july': 'julio',
    'august': 'agosto',
    'september': 'septiembre',
    'october': 'octubre',
    'november': 'noviembre',
    'december': 'diciembre'
}
# Genero la columna con los meses en castellano 
horas_autorizadas['mes_esp'] = horas_autorizadas['mes_ing'].map(dic_meses)
# Ordeno el DF
horas_autorizadas = horas_autorizadas[['no', 'nombre', 'localidad', 'zona', 'gerente regional',  'mes_esp', 'horario_domingo', 'horario_anterior', 'horas_autorizadas', 'cantidad_domingos', 'horas_por_domingo', 'horas_promedio_abril_y_mayo', 'crecimiento_horas_domingo', 'horas_adicionales_por_domingo', 'horas_adicionales_total_mes', 'vct', 'vst', 'ratio_margen','margen', 'gastos_variables', 'costo_hora_hombre', 'gasto_de_personal', 'roc']]
# Genero un bucle para redondear todos los valores numericos, excepto los porcentajes 
for col in horas_autorizadas.select_dtypes('number').columns:

    if col == 'ratio_margen' or col == 'crecimiento_horas_domingo':
        horas_autorizadas[col] = horas_autorizadas[col].round(2)
        
    else:
        horas_autorizadas[col] = horas_autorizadas[col].round(1)

# Pivoteo la Informacion para mostrarla en formato LONG
horas_autorizadas = horas_autorizadas.pivot_table(values=['horas_autorizadas', 'cantidad_domingos', 'horas_por_domingo', 'horas_promedio_abril_y_mayo', 'crecimiento_horas_domingo', 'horas_adicionales_por_domingo', 'horas_adicionales_total_mes', 'vct', 'vst', 'ratio_margen','margen', 'gastos_variables', 'costo_hora_hombre', 'gasto_de_personal', 'roc'], columns='mes_esp', index=['no', 'nombre', 'localidad', 'zona', 'gerente regional', 'horario_domingo', 'horario_anterior'], aggfunc='sum')

# Ordeno las columans de orden mayor y de orden menor
orden_columnas_primer_nivel = [
    'horas_autorizadas', 'cantidad_domingos', 'horas_por_domingo',
    'horas_promedio_abril_y_mayo', 'crecimiento_horas_domingo',
    'horas_adicionales_por_domingo', 'horas_adicionales_total_mes',
    'vct', 'vst', 'ratio_margen', 'margen', 'gastos_variables',
    'costo_hora_hombre', 'gasto_de_personal', 'roc'
    ]
horas_autorizadas = horas_autorizadas.reindex(orden_columnas_primer_nivel, level=0, axis=1)
orden_columnas_segundo_nivel = ['abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
horas_autorizadas = horas_autorizadas.reindex(orden_columnas_segundo_nivel, level=1, axis=1)

horas_autorizadas = horas_autorizadas.reset_index()

#--- FORMATEO FINAL DE COLUMNAS ---
# Formatear nombres del MultiIndex (capitalizar y reemplazar guiones bajos)
horas_autorizadas.columns = pd.MultiIndex.from_tuples([
(
str(col[0]).replace('_', ' ').capitalize(),
str(col[1]).replace('_', ' ').capitalize()
)
for col in horas_autorizadas.columns
])

ultimo_mes_variable = domingos_por_mes['mes'].to_list()[-1]

horas_autorizadas.sort_values(by=('Roc', dic_meses[ultimo_mes_variable].capitalize()))

# Exporto el resultado final a EXCEL al path Results
horas_autorizadas.reset_index().to_excel('results/analisis_tiendas_horario_extendido_domingos_express.xlsx', index_label='indice', sheet_name='data')

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_3404\4248167377.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domingos['no'] = df_domingos['no'].astype(int)


In [79]:
ultimo_mes_variable = domingos_por_mes['mes'].to_list()[-1]

horas_autorizadas.sort_values(by=('Roc', dic_meses[ultimo_mes_variable].capitalize()), ascending=True)

No                     Nombre Localidad Zona  Gerente regional  \
                                                                      
58  481      Boulogne Sur Mer 1398       GBA    2     Marcela Muñoz   
31  435               Ruta 26 162        GBA    1     Nestor Flores   
46  461                  Salta 160       GBA    3  Jaquelina Correa   
81  575         Av. Directorio 483      CABA   11      Romina Ponce   
61  485  AV. Pte. J D Perón  23866       GBA    2     Marcela Muñoz   
..  ...                        ...       ...  ...               ...   
74  537               Charcas 4367      CABA    8     Matias Flores   
91  697                 Caboto 420      CABA   11      Romina Ponce   
65  493               Charcas 3940      CABA    9  Ana Amelia Cohen   
70  512       Francisco Ramirez 43       GBA    1     Nestor Flores   
18  388            Alte. Segui 849       GBA    3  Jaquelina Correa   

   Horario domingo Horario anterior Horas autorizadas                ...  \
                                                Abril   Mayo  Junio  ...   
58    8:00 - 21:00    09:00 - 20:30              97.0   75.0  218.0  ...   
31    8:00 - 21:00     9:00 - 21:00             105.0   69.5  137.0  ...   
46    8:00 - 21:00    10:00 - 19:00              67.0   52.0  137.0  ...   
81    8:00 - 22:00    11:00 - 20:00              72.5   40.0  119.5  ...   
61    8:00 - 21:00    10:30 - 20:00              58.0   50.0  138.0  ...   
..             ...              ...               ...    ...    ...  ...   
74    8:00 - 22:00    11:00 - 20:00              96.5   82.0  159.0  ...   
91    8:00 - 21:00    10:00 - 19:00              64.0   48.5  150.5  ...   
65    8:00 - 22:00    11:00 - 20:00             104.5   72.0  191.0  ...   
70    7:00 - 21:00     9:00 - 18:00             338.0  229.0  440.5  ...   
18    8:00 - 21:00    10:00 - 19:00             400.5  300.5  551.5  ...   

   Gasto de personal                                        Roc             \
               Junio     Julio     Agosto Septiembre      Abril       Mayo   
58         -486992.9 -428377.5  -317316.7  -393120.1   -47597.5    52375.0   
31         -119523.2 -140092.3   -25444.1  -141802.9   -74983.1    76126.9   
46         -276685.7 -196597.2   -68950.4  -256707.8    26475.4    72494.1   
81         -209368.2  -99407.4   -49788.8  -172601.1    -3529.0   153069.9   
61         -332483.4 -328239.0  -238161.9  -328239.0   -18864.3    18864.3   
..               ...       ...        ...        ...        ...        ...   
74         -203166.7 -172809.4   -61870.6  -131705.1    50382.3    57741.3   
91         -350175.4 -138672.9  -131861.6  -264421.7    33414.4    89717.6   
65         -352942.6 -299848.1  -178002.0  -275356.4    -9024.2    89475.3   
70         -372494.9 -109369.0  1533242.7  1226594.2  1743245.3  2542386.4   
18         -515526.1 -145111.1  1991640.1  1593312.1  1884153.0  2410737.6   

                                                
        Junio      Julio     Agosto Septiembre  
58  -298495.4  -224926.4  -126962.2  -181097.0  
31   -79210.2  -110796.6    65095.7   -96357.9  
46   -32982.0    -2111.4   165469.4   -78259.7  
81   418932.0   260788.5    93490.4   -57153.2  
61   -93413.5  -102857.9    41885.9   -52724.6  
..        ...        ...        ...        ...  
74   748555.6   921274.6  1134485.1  1223597.8  
91   948928.8   861613.7  1422406.8  1301452.3  
65  1510533.6  1366394.0  1542249.8  1659327.0  
70  1130315.6  1693958.2  3993326.8  2553837.1  
18  2071156.3  1897179.8  4536610.3  3995862.2  

[99 rows x 97 columns]

In [69]:
horas_autorizadas.columns

MultiIndex([(                           'No',           ''),
            (                       'Nombre',           ''),
            (                    'Localidad',           ''),
            (                         'Zona',           ''),
            (             'Gerente regional',           ''),
            (              'Horario domingo',           ''),
            (             'Horario anterior',           ''),
            (            'Horas autorizadas',      'Abril'),
            (            'Horas autorizadas',       'Mayo'),
            (            'Horas autorizadas',      'Junio'),
            (            'Horas autorizadas',      'Julio'),
            (            'Horas autorizadas',     'Agosto'),
            (            'Horas autorizadas', 'Septiembre'),
            (            'Cantidad domingos',      'Abril'),
            (            'Cantidad domingos',       'Mayo'),
            (            'Cantidad domingos',      'Junio'),
            (           

In [55]:
ventas_horario_extendido

,no,mes,vct
0,200,april,1053180.04
1,200,august,3249136.54
2,200,july,2185956.55
3,200,june,2406038.41
4,200,may,641432.31
...,...,...,...
575,854,august,3902629.26
576,854,july,2717838.07
577,854,june,2778483.50
578,854,may,4270.50


In [56]:
# Confirmo que la seleccion de vct en horario extendido es correcta. Lau tiene otros numeros en su reporte
ventas_horario_extendido[ventas_horario_extendido['no'] == 200]

,no,mes,vct
0,200,april,1053180.04
1,200,august,3249136.54
2,200,july,2185956.55
3,200,june,2406038.41
4,200,may,641432.31
5,200,september,2802756.66
